# Fetch Municipality and Groups Metadata from Kolada API

This notebook fetches municipality, municipality groups, and organizational units metadata from the Kolada API and stores them in the Lakehouse.

**API Endpoints:**
- `http://api.kolada.se/v2/municipality`
- `http://api.kolada.se/v2/municipality_groups`
- `http://api.kolada.se/v2/ou`
- `http://api.kolada.se/v2/kpi_groups`

**Output:** Multiple metadata tables in Lakehouse

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime
import time

In [ ]:
# Configuration
API_BASE_URL = "http://api.kolada.se/v2"
PER_PAGE = 5000

In [ ]:
def fetch_metadata(endpoint, table_name):
    """
    Generic function to fetch metadata from Kolada API with pagination.
    
    Args:
        endpoint: API endpoint (e.g., 'municipality', 'kpi_groups')
        table_name: Name for the output table
        
    Returns:
        DataFrame: Pandas DataFrame with the fetched data
    """
    print(f"\n{'='*60}")
    print(f"Fetching {endpoint} metadata")
    print(f"{'='*60}")
    
    all_data = []
    url = f"{API_BASE_URL}/{endpoint}?per_page={PER_PAGE}"
    
    page_count = 0
    
    while url:
        try:
            print(f"Fetching page {page_count + 1}...")
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            
            data = response.json()
            
            if 'values' in data:
                all_data.extend(data['values'])
                print(f"  Retrieved {len(data['values'])} items (Total: {len(all_data)})")
            
            # Check for next page
            url = data.get('next_page', None)
            page_count += 1
            
            # Be nice to the API
            if url:
                time.sleep(0.5)
                
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            break
    
    print(f"Total items fetched: {len(all_data)}")
    
    if all_data:
        df = pd.DataFrame(all_data)
        df['ingestion_timestamp'] = datetime.now()
        df['source_system'] = 'Kolada API'
        return df
    else:
        return None

In [ ]:
def save_to_lakehouse(df, table_name):
    """
    Save DataFrame to Lakehouse as Delta table.
    
    Args:
        df: Pandas DataFrame to save
        table_name: Name of the table
    """
    if df is None or df.empty:
        print(f"  ✗ No data to save for {table_name}")
        return
    
    spark_df = spark.createDataFrame(df)
    spark_df.write.mode("overwrite").format("delta").saveAsTable(table_name)
    
    print(f"  ✓ Successfully wrote {len(df)} rows to {table_name}")

In [ ]:
# Fetch Municipality metadata
df_municipality = fetch_metadata('municipality', 'dMunicipality')
if df_municipality is not None:
    # Add surrogate key
    df_municipality['municipality_key'] = range(1, len(df_municipality) + 1)
    
    print(f"\nDataFrame shape: {df_municipality.shape}")
    print(f"Columns: {list(df_municipality.columns)}")
    display(df_municipality.head())
    save_to_lakehouse(df_municipality, 'dMunicipality')

In [ ]:
# Fetch Municipality Groups metadata
df_municipality_groups = fetch_metadata('municipality_groups', 'dMunicipalityGroup')
if df_municipality_groups is not None:
    print(f"\nDataFrame shape: {df_municipality_groups.shape}")
    print(f"Columns: {list(df_municipality_groups.columns)}")
    
    # Flatten the members array if it exists
    if 'members' in df_municipality_groups.columns:
        # Store the main groups table
        df_groups_main = df_municipality_groups[['id', 'title', 'ingestion_timestamp', 'source_system']].copy()
        save_to_lakehouse(df_groups_main, 'dMunicipalityGroup')
        
        # Create a separate table for group members
        members_list = []
        for idx, row in df_municipality_groups.iterrows():
            if row['members'] and isinstance(row['members'], list):
                for member in row['members']:
                    members_list.append({
                        'group_id': row['id'],
                        'group_title': row['title'],
                        'member_id': member.get('id'),
                        'member_title': member.get('title'),
                        'ingestion_timestamp': row['ingestion_timestamp'],
                        'source_system': row['source_system']
                    })
        
        if members_list:
            df_members = pd.DataFrame(members_list)
            # Add surrogate key lookup for member_id
            if df_municipality is not None:
                muni_lookup = df_municipality.set_index('id')['municipality_key'].to_dict()
                df_members['municipality_key'] = df_members['member_id'].map(muni_lookup)
            save_to_lakehouse(df_members, 'brMunicipalityGroupMember')
    else:
        save_to_lakehouse(df_municipality_groups, 'dMunicipalityGroup')
    
    display(df_municipality_groups.head())

In [ ]:
# Fetch KPI Groups metadata
df_kpi_groups = fetch_metadata('kpi_groups', 'dKpiGroup')
if df_kpi_groups is not None:
    print(f"\nDataFrame shape: {df_kpi_groups.shape}")
    print(f"Columns: {list(df_kpi_groups.columns)}")
    
    # Flatten the members array if it exists
    if 'members' in df_kpi_groups.columns:
        # Store the main groups table
        df_kpi_groups_main = df_kpi_groups[['id', 'title', 'ingestion_timestamp', 'source_system']].copy()
        save_to_lakehouse(df_kpi_groups_main, 'dKpiGroup')
        
        # Create a separate table for group members
        kpi_members_list = []
        for idx, row in df_kpi_groups.iterrows():
            if row['members'] and isinstance(row['members'], list):
                for member in row['members']:
                    kpi_members_list.append({
                        'group_id': row['id'],
                        'group_title': row['title'],
                        'kpi_id': member.get('id'),
                        'kpi_title': member.get('title'),
                        'ingestion_timestamp': row['ingestion_timestamp'],
                        'source_system': row['source_system']
                    })
        
        if kpi_members_list:
            df_kpi_members = pd.DataFrame(kpi_members_list)
            # Note: kpi_key will be looked up from dKpi table (run notebook 01 first)
            save_to_lakehouse(df_kpi_members, 'brKpiGroupMember')
    else:
        save_to_lakehouse(df_kpi_groups, 'dKpiGroup')
    
    display(df_kpi_groups.head())

In [ ]:
# Fetch Organizational Units metadata
df_ou = fetch_metadata('ou', 'dOrganizationalUnit')
if df_ou is not None:
    print(f"\nDataFrame shape: {df_ou.shape}")
    print(f"Columns: {list(df_ou.columns)}")

    # Add 'No OU' placeholder rows for each municipality
    if df_municipality is not None:
        print(f"\nAdding 'No OU' placeholder rows...")
        df_no_ou = pd.DataFrame({
            'id': 'NO_OU_' + df_municipality['id'].astype(str),
            'municipality': df_municipality['id'].values,
            'title': 'No OU',
            'ingestion_timestamp': df_ou['ingestion_timestamp'].iloc[0],
            'source_system': 'Kolada API'
        })
        df_ou = pd.concat([df_ou, df_no_ou], ignore_index=True)
        print(f"  Added {len(df_no_ou)} 'No OU' placeholder rows")

    # Add surrogate key after adding placeholders
    df_ou['ou_key'] = range(1, len(df_ou) + 1)

    display(df_ou.head())
    save_to_lakehouse(df_ou, 'dOrganizationalUnit')

In [ ]:
# Summary
print("\n" + "="*60)
print("SUMMARY")
print("="*60)

if df_municipality is not None:
    print(f"\nMunicipalities: {len(df_municipality)} total")
    print(f"  By type: {df_municipality['type'].value_counts().to_dict()}")

if df_municipality_groups is not None:
    print(f"\nMunicipality Groups: {len(df_municipality_groups)} total")

if df_kpi_groups is not None:
    print(f"\nKPI Groups: {len(df_kpi_groups)} total")

if df_ou is not None:
    print(f"\nOrganizational Units: {len(df_ou)} total")